In [ ]:
import hopsworks
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt

In [2]:
load_dotenv()
project_name = os.getenv("HOPSWORKS_PROJECT")
api_key = os.getenv("HOPSWORKS_API_KEY")
test_start_string = os.getenv("TEST_START_DATE")
test_start_date = pd.to_datetime(test_start_string).date()

project = hopsworks.login(project=project_name, api_key_value=api_key)
fs = project.get_feature_store()

2026-01-06 09:39:24,140 INFO: Initializing external client
2026-01-06 09:39:24,141 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-06 09:39:24,812 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-06 09:39:25,654 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271989


In [3]:
print(test_start_date)
print(type(test_start_date))

2025-12-30
<class 'datetime.date'>


In [4]:
vehicle_fg = fs.get_or_create_feature_group(name="vehicle_trip_agg_fg", version=2)
vehicle_df = vehicle_fg.read()

weather_fg = fs.get_or_create_feature_group(name="weather_hourly_fg", version=1)
weather_df = weather_fg.read()

holiday_fg = fs.get_or_create_feature_group(name="swedish_holidays_fg", version=1)
holiday_df = holiday_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (87.11s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.90s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.65s) 


In [5]:
vehicle_df["_date"] = vehicle_df["window_start"].dt.date
print(vehicle_df["_date"].head)

<bound method NDFrame.head of 0          2025-11-24
1          2025-11-24
2          2025-11-24
3          2025-11-24
4          2025-11-24
              ...    
6253617    2025-12-09
6253618    2025-12-09
6253619    2025-12-09
6253620    2025-12-09
6253621    2025-12-09
Name: _date, Length: 6253622, dtype: object>


In [6]:
vehicle_df["date"] = vehicle_df["_date"]
weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.date
holiday_df["date"] = pd.to_datetime(holiday_df["date"]).dt.date

In [7]:
# Features to use for training
VEHICLE_FEATURES = [
    "trip_id",
    "vehicle_id",
    "avg_speed",
    "max_speed",
    "speed_std",
    "n_positions",
    "lat_mean",
    "lon_mean",
    "hour",
    "day_of_week",
]

WEATHER_FEATURES = [
    "temperature_2m",
    "precipitation",
    "cloud_cover",
    "wind_speed_10m",
    "snowfall",
    "rain"
]

HOLIDAY_FEATURES = [
    "is_work_free",
    "is_red_day",
    "is_day_before_holiday",
]

# Target variable
TARGET = "occupancy_mode"

In [8]:
vehicle_df["window_start"] = vehicle_df["window_start"].dt.tz_convert(None)

print(vehicle_df["window_start"].head)

<bound method NDFrame.head of 0         2025-11-24 15:59:00
1         2025-11-24 10:39:00
2         2025-11-24 17:02:00
3         2025-11-24 10:25:00
4         2025-11-24 04:32:00
                  ...        
6253617   2025-12-09 06:59:00
6253618   2025-12-09 04:49:00
6253619   2025-12-09 05:56:00
6253620   2025-12-09 15:24:00
6253621   2025-12-09 13:59:00
Name: window_start, Length: 6253622, dtype: datetime64[us]>


In [ ]:
# Prepare weather and holiday data for joining
weather_df["date"] = pd.to_datetime(weather_df["date"]).dt.date
holiday_df["date"] = pd.to_datetime(holiday_df["date"]).dt.date

# Weather has hourly data, so join on date + hour
weather_df["_date"] = weather_df["date"]
weather_df["_hour"] = weather_df["hour"]

# Create merged df - JOIN ON DATE AND HOUR (not window_start vs date!)
# This preserves all 6.2M rows instead of losing 99.7% of data
merged_df = (
    vehicle_df[["window_start", "date", "hour", "occupancy_mode"] + VEHICLE_FEATURES]
    .merge(
        weather_df[["_date", "_hour"] + WEATHER_FEATURES],
        left_on=["date", "hour"],
        right_on=["_date", "_hour"],
        how="left"
    )
    .merge(
        holiday_df[["date"] + HOLIDAY_FEATURES],
        on="date",
        how="left"
    )
)

# Drop helper columns
merged_df = merged_df.drop(columns=["_date", "_hour"], errors="ignore")

# Sort by time for lag creation
merged_df = merged_df.sort_values(by=["window_start"])

for col in HOLIDAY_FEATURES:
    if col in merged_df.columns:
        merged_df = merged_df.dropna(subset=HOLIDAY_FEATURES)
        merged_df[col] = merged_df[col].astype(int)


In [13]:
print(merged_df.columns)
print(merged_df["window_start"].isna().sum())
merged_df = merged_df.dropna(subset=["window_start"])

Index(['trip_id', 'vehicle_id', 'window_start', 'occupancy_mode', 'trip_id',
       'vehicle_id', 'avg_speed', 'max_speed', 'speed_std', 'n_positions',
       'lat_mean', 'lon_mean', 'hour', 'day_of_week', 'date_x',
       'temperature_2m', 'precipitation', 'cloud_cover', 'wind_speed_10m',
       'snowfall', 'rain', 'date_y', 'is_work_free', 'is_red_day',
       'is_day_before_holiday'],
      dtype='object')
0


In [14]:
print(merged_df["window_start"].dtype)
print(merged_df["window_start"].isna().sum())
print(f"Rows after merge: {len(merged_df)}")


datetime64[us]
0


In [15]:
# Drop the temporary date column used for joining
merged_df = merged_df.drop(columns=["date"], errors="ignore")
print(merged_df.columns)


datetime64[ns]
Index(['trip_id', 'vehicle_id', 'window_start', 'occupancy_mode', 'trip_id',
       'vehicle_id', 'avg_speed', 'max_speed', 'speed_std', 'n_positions',
       'lat_mean', 'lon_mean', 'hour', 'day_of_week', 'temperature_2m',
       'precipitation', 'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain',
       'is_work_free', 'is_red_day', 'is_day_before_holiday'],
      dtype='object')


In [16]:
merged_df["window_start"] = pd.to_datetime(merged_df["window_start"], errors="coerce")
merged_df = merged_df.dropna(subset=["window_start"])

In [ ]:
# LAGS = [1, 2, 3]

# # Lag features for target variable
# for lag in LAGS:
#     merged_df[f"{TARGET}_lag_{lag}"] = merged_df.groupby("vehicle_id")[TARGET].shift(lag)
    
# lag_cols = [f"{TARGET}_lag_{lag}" for lag in LAGS]
# merged_df = merged_df.dropna(subset=lag_cols)

# merged_df = merged_df[merged_df['trip_id'].apply(lambda x: str(x).strip() != '')]
# merged_df = merged_df[merged_df['vehicle_id'].apply(lambda x: str(x).strip() != '')]

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
print(merged_df.columns)
print(type(merged_df['trip_id']))

merged_df['trip_id'] = pd.to_numeric(merged_df['trip_id'], errors='coerce')
merged_df = merged_df.dropna(subset=['trip_id'])

merged_df['vehicle_id'] = pd.to_numeric(merged_df['vehicle_id'], errors='coerce')
merged_df = merged_df.dropna(subset=['vehicle_id'])

# Make sure id columns are integers
merged_df['trip_id'] = merged_df['trip_id'].astype(int)
merged_df['vehicle_id'] = merged_df['vehicle_id'].astype(int)

merged_fg = fs.get_or_create_feature_group(
    name="merged_fg",
    description="Vehicle, weather, holiday, traffic features with occupancy",
    version=1,
    primary_key=["trip_id"],
    event_time="window_start"
)

print("Rows to insert:", len(merged_df))
print(merged_df.head())

merged_fg.insert(merged_df)

Index(['trip_id', 'vehicle_id', 'window_start', 'occupancy_mode', 'avg_speed',
       'max_speed', 'speed_std', 'n_positions', 'lat_mean', 'lon_mean', 'hour',
       'day_of_week', 'temperature_2m', 'precipitation', 'cloud_cover',
       'wind_speed_10m', 'snowfall', 'rain', 'is_work_free', 'is_red_day',
       'is_day_before_holiday'],
      dtype='object')
<class 'pandas.core.series.Series'>
Rows to insert: 17765
                   trip_id        vehicle_id window_start  occupancy_mode  \
4521114  55700000081734464  9031005900306033   2025-11-24               0   
4451424  55700000080524256  9031005920804453   2025-11-24               0   
67636    55700000075898632  9031005920703441   2025-11-24               0   
4521115  55700000081734464  9031005900306033   2025-11-24               0   
4451413  55700000080524256  9031005920804453   2025-11-24               0   

         avg_speed  max_speed  speed_std  n_positions   lat_mean   lon_mean  \
4521114        0.0        0.0        0.

Uploading Dataframe: 100.00% |██████████| Rows 17765/17765 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: merged_fg_1_offline_fg_materialization


Uploading Dataframe: 12.51% |█▎        | Rows 2222/17765 | Elapsed Time: 00:20 | Remaining Time: 08:12

Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271989/jobs/named/merged_fg_1_offline_fg_materialization/executions


(Job('merged_fg_1_offline_fg_materialization', 'SPARK'), None)

%6|1767689746.662|FAIL|rdkafka#producer-5| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 56497ms in state UP)
%6|1767689746.950|FAIL|rdkafka#producer-5| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50800ms in state UP)
%6|1767689797.144|FAIL|rdkafka#producer-5| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 100993ms in state UP)
%6|1767689847.593|FAIL|rdkafka#producer-5| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50003ms in state UP, 1 identical error(s) suppressed)


In [31]:
# lag_features = [f"{TARGET}_lag_{lag}" for lag in LAGS]

selected_features = merged_fg.select(
    ["window_start", "occupancy_mode"] + VEHICLE_FEATURES + WEATHER_FEATURES + HOLIDAY_FEATURES
)

feature_view_name = "occupancy_fv"
feature_view_version = 1

feature_view = fs.get_or_create_feature_view(
    name=feature_view_name,
    description="Vehicle, weather, holiday features with lagged occupancy target",
    version=feature_view_version,
    labels=[TARGET],
    query=selected_features
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1271989/fs/1258587/fv/occupancy_fv/version/1


In [32]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_start=test_start_date
)

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.43s) 
2026-01-06 09:59:20,483 WARNING: VersionWarning: Incremented version to `1`.

Train samples: 660, Test samples: 131


In [33]:
print(X_train.columns)
print(X_test.columns)

Index(['window_start', 'trip_id', 'vehicle_id', 'avg_speed', 'max_speed',
       'speed_std', 'n_positions', 'lat_mean', 'lon_mean', 'hour',
       'day_of_week', 'temperature_2m', 'precipitation', 'cloud_cover',
       'wind_speed_10m', 'snowfall', 'rain', 'is_work_free', 'is_red_day',
       'is_day_before_holiday'],
      dtype='object')
Index(['window_start', 'trip_id', 'vehicle_id', 'avg_speed', 'max_speed',
       'speed_std', 'n_positions', 'lat_mean', 'lon_mean', 'hour',
       'day_of_week', 'temperature_2m', 'precipitation', 'cloud_cover',
       'wind_speed_10m', 'snowfall', 'rain', 'is_work_free', 'is_red_day',
       'is_day_before_holiday'],
      dtype='object')


In [34]:
X_train.describe()

,trip_id,vehicle_id,avg_speed,max_speed,speed_std,n_positions,lat_mean,lon_mean,hour,day_of_week,temperature_2m,precipitation,cloud_cover,wind_speed_10m,snowfall,rain,is_work_free,is_red_day,is_day_before_holiday
count,6.600000e+02,6.600000e+02,660.000000,660.000000,658.000000,660.000000,660.000000,660.000000,660.0,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,5.570001e+16,9.031006e+15,4.959702,6.819091,1.289427,53.209091,58.481246,15.841689,0.0,4.121212,3.478333,0.045455,73.275758,14.611667,0.000106,0.045303,0.457576,0.331818,0.013636
std,3.942894e+10,9.110827e+06,6.364188,7.715031,1.592389,10.500896,0.112876,0.329854,0.0,1.885293,2.753941,0.174889,40.658648,5.941848,0.002725,0.174625,0.498575,0.471223,0.116064
min,5.570000e+16,9.031006e+15,0.000000,0.000000,0.000000,1.000000,58.000500,15.046250,0.0,0.000000,-3.900000,0.000000,0.000000,2.600000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.570000e+16,9.031006e+15,0.000000,0.000000,0.000000,52.000000,58.417168,15.642227,0.0,3.000000,1.600000,0.000000,42.500000,9.900000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.570000e+16,9.031006e+15,2.821065,6.100000,0.762339,56.000000,58.426861,15.655000,0.0,5.000000,4.200000,0.000000,100.000000,14.300000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.570000e+16,9.031006e+15,8.291928,10.800000,2.137056,59.000000,58.584944,16.189955,0.0,6.000000,5.200000,0.000000,100.000000,18.725000,0.000000,0.000000,1.000000,1.000000,0.000000
max,5.570020e+16,9.031006e+15,43.940386,44.400002,8.337641,61.000000,58.710456,16.558377,0.0,6.000000,9.200000,1.300000,100.000000,33.100000,0.070000,1.300000,1.000000,1.000000,1.000000


In [35]:
X_test.describe()

,trip_id,vehicle_id,avg_speed,max_speed,speed_std,n_positions,lat_mean,lon_mean,hour,day_of_week,temperature_2m,precipitation,cloud_cover,wind_speed_10m,snowfall,rain,is_work_free,is_red_day,is_day_before_holiday
count,1.310000e+02,1.310000e+02,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.0,131.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.0,131.000000,131.000000,131.000000
mean,5.570002e+16,9.031006e+15,6.504716,8.681679,1.506997,57.282443,58.483579,15.822426,0.0,4.442748,-3.979487,0.369231,99.961538,14.950000,0.258462,0.0,0.419847,0.381679,0.290076
std,4.297984e+10,9.207003e+06,7.371014,7.975866,1.348230,7.193239,0.108979,0.344915,0.0,1.370908,0.955960,0.335868,0.193552,2.414822,0.235107,0.0,0.495428,0.487663,0.455539
min,5.570000e+16,9.031006e+15,0.000000,0.000000,0.000000,4.000000,58.162634,15.046633,0.0,2.000000,-5.900000,0.000000,99.000000,11.200000,0.000000,0.0,0.000000,0.000000,0.000000
25%,5.570000e+16,9.031006e+15,0.121597,0.700000,0.091699,57.000000,58.415007,15.622635,0.0,3.000000,-4.400000,0.100000,100.000000,13.300000,0.070000,0.0,0.000000,0.000000,0.000000
50%,5.570000e+16,9.031006e+15,5.355000,8.600000,1.318701,59.000000,58.426935,15.656965,0.0,5.000000,-3.800000,0.300000,100.000000,14.400000,0.210000,0.0,0.000000,0.000000,0.000000
75%,5.570000e+16,9.031006e+15,8.696491,11.250000,2.308260,60.000000,58.584856,16.183746,0.0,6.000000,-3.200000,0.500000,100.000000,17.300000,0.350000,0.0,1.000000,1.000000,1.000000
max,5.570020e+16,9.031006e+15,43.465001,44.700001,6.221950,81.000000,58.705970,16.564897,0.0,6.000000,-2.500000,1.500000,100.000000,18.700000,1.050000,0.0,1.000000,1.000000,1.000000


%6|1767689999.332|FAIL|rdkafka#producer-5| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50124ms in state UP, 1 identical error(s) suppressed)


In [36]:
# Dropping features that had less than 0.02 in feature importance in a test run, or has no predictive power
X_features = X_train.drop(columns=['speed_std', 'avg_speed'])             # 'window_start'
X_test_features = X_test.drop(columns=['speed_std', 'avg_speed'])         # 'window_start'

In [ ]:
X_features

,window_start,trip_id,vehicle_id,max_speed,n_positions,lat_mean,lon_mean,hour,day_of_week,temperature_2m,precipitation,cloud_cover,wind_speed_10m,snowfall,rain,is_work_free,is_red_day,is_day_before_holiday
1,2025-12-20 00:00:00+00:00,55700000080481272,9031005921001421,5.0,56,58.322589,15.132894,0,5,7.7,0.0,100.0,21.4,0.0,0.0,1,1,0
2,2025-12-06 00:00:00+00:00,55700000074081136,9031005920505748,15.8,58,58.389378,15.677108,0,5,4.4,0.0,100.0,11.1,0.0,0.0,1,1,0
5,2025-12-29 00:00:00+00:00,55700000082794496,9031005920505505,0.0,4,58.426601,15.643596,0,0,-1.9,0.0,2.0,24.5,0.0,0.0,0,0,1
6,2025-12-20 00:00:00+00:00,55700000080650328,9031005920505509,8.1,56,58.400243,15.621281,0,5,6.2,0.0,95.0,20.3,0.0,0.0,1,1,0
8,2025-12-03 00:00:00+00:00,55700000080518880,9031005920804460,0.0,55,58.203941,16.000764,0,2,3.2,0.0,100.0,8.1,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2025-12-06 00:00:00+00:00,55700000075973832,9031005920505735,1.4,58,58.417858,15.622246,0,5,5.4,0.0,100.0,9.9,0.0,0.0,1,1,0
787,2025-12-06 00:00:00+00:00,55700000081780608,9031005900306692,9.7,58,58.666783,16.181294,0,5,4.9,0.8,100.0,7.1,0.0,0.8,1,1,0
788,2025-12-04 00:00:00+00:00,55700000075135112,9031005920505507,0.0,59,58.426632,15.643848,0,3,3.2,0.0,100.0,8.6,0.0,0.0,0,0,0
789,2025-12-14 00:00:00+00:00,55700000081794024,9031005900306729,16.4,53,58.642663,16.165292,0,6,6.9,0.1,100.0,20.6,0.0,0.1,0,0,0


In [39]:
y_train

,occupancy_mode
1,0
2,1
5,0
6,1
8,0
...,...
786,1
787,0
788,0
789,0


In [40]:
from xgboost import XGBClassifier
from scipy.stats import randint, uniform

XGBOOST_PARAMS = {
    "tree_method": "hist",
    "enable_categorical": True,
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 200,
    "subsample": 0.7,
    "colsample_bytree": 0.8,
    "min_child_weight": 1,
    "gamma": 0.1,
    "objective": "multi:softprob",
    "num_class": 7,  # GTFS-RT has 7 occupancy classes (0-6)
    "random_state": 42,
}

CLASS_WEIGHT_MULTIPLIER = {
    0: 1.0,   # EMPTY (72%) - baseline
    1: 2.0,   # MANY_SEATS (26%) - slight boost
    2: 10.0,  # FEW_SEATS (1%) - significant boost
    3: 20.0,  # STANDING (0.4%) - heavy boost
    4: 25.0,  # CRUSHED_STANDING - not observed yet
    5: 30.0,  # FULL - not observed yet
    6: 1.0,   # NOT_ACCEPTING_PASSENGERS - not observed yet
}


# For hyperparameter tuning
param_dist = {
    "max_depth": randint(3, 10),
    "learning_rate": uniform(0.01, 0.3),
    "n_estimators": randint(100, 300),
    "subsample": uniform(0.6, 0.4),
    "colsample_bytree": uniform(0.6, 0.4),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0, 5),
}

In [41]:
classes = np.unique(y_train)
print(len(classes))
print(classes)
print(classes[0].dtype)

4
[0 1 2 3]
int64


In [42]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)
from sklearn.model_selection import RandomizedSearchCV

MAX_WEIGHT = 50.0
ALL_POSSIBLE_CLASSES = np.array([0, 1, 2, 3, 4, 5, 6])

def compute_sample_weights(y_train):
    """Compute sample weights to handle class imbalance with custom multipliers."""
    from sklearn.utils.class_weight import compute_class_weight

    present_classes = np.unique(y_train)
    base_weights = compute_class_weight('balanced', classes=present_classes, y=y_train)
    base_weight_dict = dict(zip(present_classes, base_weights))

    # Apply additional multipliers for severe imbalance
    weight_dict = {}
    for cls in ALL_POSSIBLE_CLASSES:
        multiplier = CLASS_WEIGHT_MULTIPLIER.get(cls, 1.0)
        weight_dict[cls] = min(base_weight_dict.get(cls, 0) * multiplier, MAX_WEIGHT)  # 0 if cls not in y_train

    print(f"  Base class weights (present in y_train): {base_weight_dict}")
    print(f"  Adjusted class weights (all possible classes): {weight_dict}")

    # Assign sample weights only for rows actually in y_train
    sample_weights = np.array([weight_dict[y] for y in y_train])
    return sample_weights

def ordinal_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def train_model(X_train, y_train, use_class_weights=True):
    """Train XGBoost Classifier with optional class weighting."""
    print("\nTraining XGBoost Classifier...")
    print(f"  Parameters: {XGBOOST_PARAMS}")  

    model = XGBClassifier(**XGBOOST_PARAMS)

    if use_class_weights:
        sample_weights = compute_sample_weights(y_train)
        model.fit(X_train, y_train, sample_weight=sample_weights)
    else:
        model.fit(X_train, y_train)

    print("  Training complete!")
    return model


def train_model_tuned(X_train, y_train, use_class_weights=True, n_iter=20, cv=2):
    """Train XGBoost Classifier with optional class weighting and hyperparameter tuning."""
    print("\nStarting hyperparameter tuning...")
    
    X_train = X_train.astype('float32')

    base_model = XGBClassifier(**XGBOOST_PARAMS, use_label_encoder=False, n_jobs=-1)

    if use_class_weights:
        sample_weights = compute_sample_weights(y_train)
    else:
        sample_weights = None

    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=n_iter,
        scoring='recall_macro',  
        cv=cv,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )

    random_search.fit(X_train, y_train, sample_weight=sample_weights)

    print("\nBest hyperparameters found:", random_search.best_params_)
    best_model = random_search.best_estimator_

    return best_model

def predict_ordinal(models, X):
    p_ge_1 = models["ge_1"].predict_proba(X)[:, 1]
    p_ge_2 = models["ge_2"].predict_proba(X)[:, 1]
    p_ge_3 = models["ge_3"].predict_proba(X)[:, 1]

    preds = np.zeros(len(X), dtype=int)
    preds[p_ge_1 > 0.5] = 1
    preds[p_ge_2 > 0.5] = 2
    preds[p_ge_3 > 0.5] = 3

    return preds


def evaluate_model(model, X_test, y_test):
    """Evaluate model and return metrics."""
    print("\nEvaluating model...")

    # Get probabilities (since we use softprob objective)
    y_proba = model.predict_proba(X_test)
    y_pred = np.argmax(y_proba, axis=1)

    # Calculate metrics (weighted for class imbalance)
    accuracy = accuracy_score(y_test, y_pred)
    ordinal_mae_val = ordinal_mae(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)

    # Also calculate per-class recall (important for rare classes)
    per_class_recall = recall_score(y_test, y_pred, average=None, zero_division=0)

    metrics = {
        "accuracy": float(accuracy),
        "ordinal_mae": float(ordinal_mae_val),
        "precision_weighted": float(precision),
        "recall_weighted": float(recall),
        "f1_weighted": float(f1),
        "recall_class_0": float(per_class_recall[0]) if len(per_class_recall) > 0 else 0,
        "recall_class_1": float(per_class_recall[1]) if len(per_class_recall) > 1 else 0,
        "recall_class_2": float(per_class_recall[2]) if len(per_class_recall) > 2 else 0,
        "recall_class_3": float(per_class_recall[3]) if len(per_class_recall) > 3 else 0,
    }

    print(f"\n  Results:")
    print(f"    Accuracy:  {accuracy:.4f}")
    print(f"    Precision: {precision:.4f} (weighted)")
    print(f"    Recall:    {recall:.4f} (weighted)")
    print(f"    F1 Score:  {f1:.4f} (weighted)")
    print(f"\n  Per-class Recall (critical for rare classes):")
    class_names = ["EMPTY", "MANY_SEATS", "FEW_SEATS", "STANDING"]
    for i, name in enumerate(class_names):
        if i < len(per_class_recall):
            print(f"    Class {i} ({name}): {per_class_recall[i]:.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n  Confusion Matrix:")
    print(cm)

    # Classification report
    print(f"\n  Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    return metrics, y_pred


def plot_feature_importance(model, feature_names, save_path=None):
    """Plot and optionally save feature importance."""
    importance = model.feature_importances_

    # Sort by importance
    indices = np.argsort(importance)[::-1]
    sorted_features = [feature_names[i] for i in indices]
    sorted_importance = importance[indices]

    print("\n  Feature Importance (gain):")
    for feat, imp in zip(sorted_features, sorted_importance):
        print(f"    {feat}: {imp:.4f}")

    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(sorted_features)), sorted_importance[::-1])
    plt.yticks(range(len(sorted_features)), sorted_features[::-1])
    plt.xlabel("Feature Importance (Gain)")
    plt.title("XGBoost Feature Importance - Occupancy Prediction")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"  Saved feature importance plot to {save_path}")

    plt.close()


def save_model_local(model, model_dir):
    """Save model to local directory."""
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "model.json")
    model.save_model(model_path)
    print(f"  Model saved to {model_path}")
    return model_path

In [43]:
print(y_train.columns)
print(X_features.columns)

print(y_test.columns)

Index(['occupancy_mode'], dtype='object')
Index(['window_start', 'trip_id', 'vehicle_id', 'max_speed', 'n_positions',
       'lat_mean', 'lon_mean', 'hour', 'day_of_week', 'temperature_2m',
       'precipitation', 'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain',
       'is_work_free', 'is_red_day', 'is_day_before_holiday'],
      dtype='object')
Index(['occupancy_mode'], dtype='object')


In [45]:
print(X_features.columns)
print(X_test_features.columns)

Index(['window_start', 'trip_id', 'vehicle_id', 'max_speed', 'n_positions',
       'lat_mean', 'lon_mean', 'hour', 'day_of_week', 'temperature_2m',
       'precipitation', 'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain',
       'is_work_free', 'is_red_day', 'is_day_before_holiday'],
      dtype='object')
Index(['window_start', 'trip_id', 'vehicle_id', 'max_speed', 'n_positions',
       'lat_mean', 'lon_mean', 'hour', 'day_of_week', 'temperature_2m',
       'precipitation', 'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain',
       'is_work_free', 'is_red_day', 'is_day_before_holiday'],
      dtype='object')


In [46]:
y_train_series = y_train['occupancy_mode']
y_train_series = y_train_series.astype(int)

print(y_train_series)

1      0
2      1
5      0
6      1
8      0
      ..
786    1
787    0
788    0
789    0
790    1
Name: occupancy_mode, Length: 660, dtype: int64


In [47]:
y_test_series = y_test['occupancy_mode']
y_test_series = y_test_series.astype(int)

print(X_test_features.shape)
print(y_test_series.shape)

(131, 18)
(131,)


In [ ]:
# X_features = X_features.drop(columns=['occupancy_mode_lag_1', 'occupancy_mode_lag_2', 'occupancy_mode_lag_3'], axis=1) 
# X_test_features = X_test_features.drop(columns=['occupancy_mode_lag_1', 'occupancy_mode_lag_2', 'occupancy_mode_lag_3', 'avg_speed', 'speed_std'], axis=1) 
# X_features = X_features.drop(columns=['avg_speed', 'speed_std'], axis=1)

In [50]:
X_test_features = X_test_features.drop(columns=['window_start'], axis=1) 
X_features = X_features.drop(columns=['window_start'], axis=1)

print(X_test_features.columns)
print(X_features.columns)

Index(['trip_id', 'vehicle_id', 'max_speed', 'n_positions', 'lat_mean',
       'lon_mean', 'hour', 'day_of_week', 'temperature_2m', 'precipitation',
       'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain', 'is_work_free',
       'is_red_day', 'is_day_before_holiday'],
      dtype='object')
Index(['trip_id', 'vehicle_id', 'max_speed', 'n_positions', 'lat_mean',
       'lon_mean', 'hour', 'day_of_week', 'temperature_2m', 'precipitation',
       'cloud_cover', 'wind_speed_10m', 'snowfall', 'rain', 'is_work_free',
       'is_red_day', 'is_day_before_holiday'],
      dtype='object')


In [ ]:
# Fill missing values
# X_train = X_train.fillna(X_train.median())
# X_test = X_test.fillna(X_test.median()) 

model_dir = "../model_plots"

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
model = train_model(X_features, y_train_series)

# Evaluate
metrics, y_pred = evaluate_model(model, X_test_features, y_test_series)

plot_feature_importance(model, X_test_features.columns.tolist(),
                                       os.path.join(model_dir, "feature_importance.png"))


Training XGBoost Classifier...
  Parameters: {'tree_method': 'hist', 'enable_categorical': True, 'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 0.7, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'objective': 'multi:softprob', 'num_class': 7, 'random_state': 42}
  Base class weights (present in y_train): {0: 0.3707865168539326, 1: 0.7932692307692307, 2: 33.0, 3: 82.5}
  Adjusted class weights (all possible classes): {0: 0.3707865168539326, 1: 1.5865384615384615, 2: 50.0, 3: 50.0, 4: 0.0, 5: 0.0, 6: 0.0}
  Training complete!

Evaluating model...

  Results:
    Accuracy:  0.7328
    Precision: 0.7428 (weighted)
    Recall:    0.7328 (weighted)
    F1 Score:  0.7365 (weighted)

  Per-class Recall (critical for rare classes):
    Class 0 (EMPTY): 0.8021
    Class 1 (MANY_SEATS): 0.5758
    Class 2 (FEW_SEATS): 0.0000

  Confusion Matrix:
[[77 19  0]
 [13 19  1]
 [ 1  1  0]]

  Classification Report:
              precision    recall  f1-score   su

In [54]:
def save_model_local(model, model_dir):
    """Save model to local directory."""
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, "model.json")
    model.save_model(model_path)
    print(f"  Model saved to {model_path}")
    return model_path

save_model_local(model, model_dir)

  Model saved to ../model_plots/model.json


'../model_plots/model.json'

In [55]:
mr = project.get_model_registry()
MODEL_NAME = "occupancy_xgboost_model"
hopsworks_model = mr.get_model(MODEL_NAME)

# Upload model directory
# hopsworks_model.feature_view = feature_view
# hopsworks_model.save(model_dir)
# Log a new version of the model directory

 # Create model in registry
hopsworks_model = mr.python.create_model(
    name="occupancy_xgboost_model_new",
    metrics=metrics,
    feature_view=feature_view,
    description="XGBoost Classifier for bus occupancy prediction (GTFS-RT classes 0-6)",
    input_example=X_test.iloc[:1].values,
)

# Upload model directory
hopsworks_model.save(model_dir)

print(f"  Model version: {hopsworks_model.version}")

2026-01-06 10:06:27,968 WARNING: VersionWarning: No version provided for getting model `occupancy_xgboost_model`, defaulting to `1`.



  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /Users/kajsalidin/Desktop/HappySardines/pipelines/../model_plots/model.json: 0.000%|          | 0/17…

Uploading /Users/kajsalidin/Desktop/HappySardines/pipelines/../model_plots/feature_importance_1.png: 0.000%|  …

Uploading /Users/kajsalidin/Desktop/HappySardines/pipelines/../model_plots/feature_importance.png: 0.000%|    …

Uploading /Users/kajsalidin/Desktop/HappySardines/pipelines/input_example.json: 0.000%|          | 0/215 elaps…

Uploading /Users/kajsalidin/Desktop/HappySardines/pipelines/model_schema.json: 0.000%|          | 0/1631 elaps…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1271989/models/occupancy_xgboost_model_new/1
  Model version: 1
